In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import astropy
import speclite
from astropy.cosmology import default_cosmology
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Observations.roman_speclite import configure_roman_filters, filter_names
from tqdm import tqdm

from mejiro.helpers import survey_sim

%matplotlib inline

In [2]:
area_hlwas = 1700.
cosmo = default_cosmology.get()

survey_area = 0.1
sky_area = Quantity(value=survey_area, unit='deg2')

In [3]:
# load Roman WFI filters
configure_roman_filters()
roman_filters = filter_names()
roman_filters.sort()
_ = speclite.filters.load_filters(*roman_filters[:8])

In [4]:
skypy_config = 'roman_hlwas.yml'

# define cuts on the intrinsic deflector and source populations (in addition to the skypy config file)
kwargs_deflector_cut = {'band': 'F106', 'band_max': 23, 'z_min': 0.01, 'z_max': 2.}
kwargs_source_cut = {'band': 'F106', 'band_max': 24, 'z_min': 0.01, 'z_max': 5.}

# create the lens population
lens_pop = LensPop(deflector_type="elliptical",  # elliptical, all-galaxies
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo)

In [5]:
num_lenses = lens_pop.deflector_number()
num_sources = lens_pop.source_number()

print(f'Number of deflectors: {num_lenses}, scaled to HLWAS ({area_hlwas} sq deg): {int((area_hlwas / survey_area) * num_lenses)}')
print(f'Number of sources: {num_sources}, scaled to HLWAS ({area_hlwas} sq deg): {int((area_hlwas / survey_area) * num_sources)}')

Number of deflectors: 1432, scaled to HLWAS (1700.0 sq deg): 24344000
Number of sources: 231604, scaled to HLWAS (1700.0 sq deg): 3937268000


In [6]:
kwargs_lens_cut = {
    'min_image_separation': 0.2,
    'max_image_separation': 5,
    'mag_arc_limit': {'F106': 25},
}

lens_population = lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cut)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mUse of unsupported NumPy function 'numpy.nan_to_num' or unsupported use of the function.
[1m
File "../../../data/bwedig/.conda/envs/mejiro/lib/python3.10/site-packages/lenstronomy/LensModel/Profiles/epl_numba.py", line 190:[0m
[1mdef alpha(x, y, b, q, t, Omega=None):
    <source elided>
    # TODO: check whether numba is active with np.nan_to_num instead of numba_util.nan_to_num
[1m    alph = (2 * b) / (1 + q) * np.nan_to_num((b / R) ** t * R / b) * Omega
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of get attribute at /data/bwedig/.conda/envs/mejiro/lib/python3.10/site-packages/lenstronomy/LensModel/Profiles/epl_numba.py (190)[0m
[1m
File "../../../data/bwedig/.conda/envs/mejiro/lib/python3.10/site-packages/lenstronomy/LensModel/Profiles/epl_numba.py", line 190:[0m
[1mdef alpha(x, y, b, q, t, Omega=None):
    <source el

NB in the current version of `slsim`, `slsim.LensPop.draw_population` is using the method `slsim.lens.validity_test()` to filter out non-detectable lenses. Here, we want to add a few more filters:

1. is the Einstein radius greater than the Sersic radius of the lensing galaxy and the lensing galaxy brighter than 15th magnitude (in F106)?
2. is the SNR of lensed images greater than 10?

In [ ]:
# set up dict to capture some information about which candidates got filtered out
filtered_sample = {}
filtered_sample['total'] = len(lens_population)
num_samples = 100
filter_1, filter_2 = 0, 0
filtered_sample['filter_1'] = []
filtered_sample['filter_2'] = []

In [ ]:
limit = None
detectable_gglenses = []

for candidate in tqdm(lens_population):
    # 1. Einstein radius and Sersic radius
    _, kwargs_params = candidate.lenstronomy_kwargs(band='F106')
    lens_mag = candidate.deflector_magnitude(band='F106')

    if kwargs_params['kwargs_lens'][0]['theta_E'] < kwargs_params['kwargs_lens_light'][0]['R_sersic'] and lens_mag < 15:
        filter_1 += 1
        if filter_1 <= num_samples:
            filtered_sample['filter_1'].append(candidate)
        continue

    # 2. SNR
    snr = survey_sim.get_snr(candidate, 'F106')

    if snr < 10:
        filter_2 += 1
        if filter_2 <= num_samples:
            filtered_sample['filter_2'].append(candidate)
        continue

    # if both criteria satisfied, consider detectable
    detectable_gglenses.append(candidate)

    # if I've imposed a limit above this loop, exit the loop
    if limit is not None and len(detectable_gglenses) == limit:
        break

print(f'{len(detectable_gglenses)} detectable lens(es)')

In [ ]:
print('Retrieving lenstronomy parameters')
dict_list = []
for gglens in tqdm(detectable_gglenses):

    # get lens params from gglens object
    kwargs_model, kwargs_params = gglens.lenstronomy_kwargs(band='F106')

    # build dicts for lens and source magnitudes
    lens_mags, source_mags = {}, {}
    for band in ['F106', 'F129', 'F184']:
        lens_mags[band] = gglens.deflector_magnitude(band)
        source_mags[band] = gglens.source_magnitude(band)

    z_lens, z_source = gglens.lens_redshift, gglens.source_redshift
    kwargs_lens = kwargs_params['kwargs_lens']

    # add additional necessary key/value pairs to kwargs_model
    kwargs_model['lens_redshift_list'] = [z_lens] * len(kwargs_lens)
    kwargs_model['source_redshift_list'] = [z_source]
    kwargs_model['cosmo'] = cosmo
    kwargs_model['z_source'] = z_source
    kwargs_model['z_source_convention'] = 5

    # create dict to pickle
    gglens_dict = {
        'kwargs_model': kwargs_model,
        'kwargs_params': kwargs_params,
        'lens_mags': lens_mags,
        'source_mags': source_mags,
        'deflector_stellar_mass': gglens.deflector_stellar_mass(),
        'deflector_velocity_dispersion': gglens.deflector_velocity_dispersion(),
        'snr': gglens.snr
    }

    dict_list.append(gglens_dict)

print('Pickling lenses...')
for i, each in tqdm(enumerate(dict_list)):
    with open(os.path.join(pickle_dir, f'skypy_output_{str(i).zfill(5)}_run{run}'), 'ab') as file:
        pickle.dump(each, file)

In [ ]:
# !jupyter nbconvert --to python hlwas_simulation.ipynb --TemplateExporter.exclude_markdown=True --TemplateExporter.exclude_output_prompt=True --TemplateExporter.exclude_input_prompt=True